Functionalities relating to applyCl. 

In [4]:
from k1lib.imports import *
def distributedInstall(): # installs the library in all nodes, should take 8-15s to complete
    base = "/home/kelvin/repos/labs/k1lib"
    applyCl.nodeIds(False) | applyCl.aS(lambda: None | cmd(f"rm -rf {base} && mkdir -p {base}") | deref()) | deref()
    None | cmd(f"cd {base} && ./distribute.sh") | ignore()
    with k1.captureStdout(True):
        applyCl.nodeIds(False) | applyCl.aS(lambda: None | cmd(f"cd {base} && unzip dist.pth && pip install .") | deref()) | ignore()

In [5]:
distributedInstall()

In [2]:
#export
__all__ = ["dummy"]
def dummy():
    """Does nothing. Only here for you to checkout the source code"""
    pass

# loadTest

In [3]:
#export
from k1lib.imports import *; import hashlib
def hashF(msg:str) -> str: m = hashlib.sha256(); m.update(f"{msg}".encode()); return k1.encode(m.digest())
def cpuHash() -> str: return None | cmd("lscpu") | head() | join("\n") | aS(hashF)
loadTestFn = "~/.k1lib/applyCl_loadTest.pth"
def load_loadTest(): return cat(loadTestFn, False) | aS(dill.loads) if os.path.exists(os.path.expanduser(loadTestFn)) else dict()
def good_loadTest(): # whether the underlying architecture has been load-tested before
    obj = load_loadTest(); return None | applyCl.aS(lambda: cpuHash()) | cut(1) | ~inSet(obj) | shape(0) == 0
def loadTestS(nodeId, cpus, hash_):
    with k1.timer() as t1:
        [nodeId]*cpus*4 | insertIdColumn(begin=False) | applyCl(lambda x: range(300_000_000) | toSum(), pre=True) | deref()
    with k1.timer() as t2:
        [nodeId]*cpus*4 | insertIdColumn(begin=False) | applyCl(lambda x: range(30_000_000) | apply(op()+2) | toSum(), pre=True) | deref()
    return [nodeId, cpus, hash_, t1(), t2()]
def loadTest():
    data = None | applyCl.aS(lambda: [os.cpu_count(), cpuHash()] | deref()) | ~apply(lambda x,y: [x,*y]) | deref()
    a = data | ~applyTh(loadTestS, timeout=3600) | deref()
    alpha1, alpha2 = a | cut(1, 3, 4) | ~apply(lambda x,y,z: [y, z]) | transpose() | toMax().all() | deref()
    obj = a | ~apply(lambda idx,cpu,h,t1,t2: [h, (alpha1/(t1), alpha2/(t2)) | toMean()]) | toDict() | deref()
    obj | aS(dill.dumps) | file(loadTestFn)
def loadTestGuard(guard=True, rounded=True): # returns Dict[nodeId, cpus]
    if guard and not good_loadTest():
        ans = input("""applyCl has not load-tested your system yet, so running
the requested operation might be unbalanced on the cluster (some
nodes finish before others, wasting computational time). Would you
like to perform a load test now? Should take 1-2 minutes. Y/n: """)
        if ans.lower()[0] == "y": loadTest()
    data = None | applyCl.aS(lambda: [cpuHash(), os.cpu_count()]) | apply(wrapList(), 0) | joinStreams().all() | deref() # List[nodeId, cpu hash, #cpus]
    obj = {**data | cut(1, 2) | apply("1", 1) | toDict(), **load_loadTest()}
    return data | lookup(obj, 1) | ~apply(lambda nodeId,mul,cpu: [nodeId, mul*cpu | (aS(round) if rounded else iden())]) | toDict()
def balancedNodeIds(): a = loadTestGuard().items(); return a | ~apply(lambda x,y: [x]*y) | joinStreams() | randomize(None) | repeatFrom() | randomize(a | cut(1) | toSum() | op()*2)

# balanceFolder

## Log scale file sizes

In [2]:
from k1lib.imports import *
base = "~/repos/labs/k1lib/k1lib/cli/test/applyCl.balance"; #base = "~/ssd2/test"
applyCl.cmd(f"rm -r {base}"); applyCl.cmd(f"mkdir -p {base}")
torch.loglinspace(10, 1e5, 20).int() | apply(lambda x: "x"*x) | insertIdColumn() | ~apply(lambda idx, contents: contents | file(f"{base}/{idx}.txt")) | deref();

In [3]:
#export
from k1lib.imports import *
getFolderSize = ls() | filt(os.path.isdir).split() | apply(lambda x: x | (tryout(0) | getFolderSize)) + apply(os.path.getsize) | toSum().all() | toSum() | deref()
getFilesInFolder = aS(os.walk) | cut(0, 2) | ungroup() | join(os.sep).all()
def getIr(base): return None | applyCl.aS(lambda: ls(base) | iden() & apply(lambda x: x | (tryout(0) | (aS(os.path.getsize) if os.path.isfile(x) else getFolderSize))) | transpose() | deref()) | ungroup(False) | insertIdColumn(True) | deref()

In [3]:
ir = getIr(base); ir | head(2)

[[0,
  '244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732',
  '/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance/15.txt',
  14385],
 [1,
  '244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732',
  '/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance/0.txt',
  11]]

ir = intermediate representation: `[id, node id, url, size]`

In [4]:
#export
def normalize(d):
    d = d | deref(); s = d | cut(1) | toSum()
    return d | apply(op()/s, 1) | sort(0, False) | deref()
@lru_cache
def statsCpu(nodeIds):
    cpu = loadTestGuard(False).items() | inSet(nodeIds, 0) | sort(0, False) | deref()
    cpuF = loadTestGuard(False).items() | inSet(nodeIds, 0) | aS(normalize); cpuF # "cpuF" = cpu fraction. List[nodeId, cpu fraction]
    return [cpu, cpuF]
@lru_cache
def statsNodeId(): return applyCl.nodeIds()
_statsS1 = statsNodeId() | apply(wrapList() | insert(0, False)) | toDict()
def stats(ir, nodeIds):
    cpu, cpuF = statsCpu(nodeIds)
    # size fraction. List[nodeId, size fraction]
    sizeF = normalize({**_statsS1, **ir | groupBy(1, True) | apply(cut(2) | toSum(), 1) | toDict()}.items()); sizeF
    return *statsCpu(nodeIds), sizeF
def scores(ir, nodeIds): cpu, cpuF, sizeF = stats(ir, nodeIds); return [cpuF, sizeF] | cut(1).all() | transpose() | ~apply(lambda c,s: s-c) | deref() # negpos
def score(ir, nodeIds): return scores(ir, nodeIds) | apply(lambda x: abs(x)**2) | toSum() # pos

In [5]:
#export
def move(ir, nA:str, nB:str, idx:int):
    ir1 = ir | deref(); ir1[idx][1] = nB; a = ir1 | pretty() | join('\n'); return ir1
def optimize(ir, nodeIds):
    cpu, cpuF, sizeF = stats(ir, nodeIds); scs = scores(ir, nodeIds); a = np.argmax(scs); b = np.argmin(scs)
    # print(scs, a, b)
    nodeIds = nodeIds | sort(None, False) | deref()
    files = {**nodeIds | apply(wrapList() | insert([], False)) | toDict(), **ir | groupBy(1, True) | toDict()}.items() | sort(0, False) | deref()
    fA = files[a]; fB = files[b] # files A. [nodeId, List[idx, url, size]]
    # print(fA); print(fB)
    nA = nodeIds[a]; nB = nodeIds[b] # A node id
    sA, sB = [fA, fB] | apply(op()[1] | cut(2) | deref() | aS(np.array, dtype=int)) # file sizes A
    # print(f"sA: {sA} {sB}")
    spA = sA.sum() - sA; spB = sB.sum() + sA # sum prime A, array[files]
    sp = spA + spB; sfA = spA/sp; sfB = spB/sp
    cA = cpu[a][1]; cB = cpu[b][1] # cpu A
    c = cA + cB; cfA = cA/c; cfB = cB/c # cpu fraction A
    # print(f"sfA: {sfA}, cfA: {cfA}")
    exp = 5 # intuition says that exp should be even. But that doesn't work. Odd values work tho, but I have no idea why
    idx = fA[1][((sfA-cfA)**exp + (sfB-cfB)**exp).argmin()][0]
    ir2 = move(ir, nA, nB, idx)
    return ir2, [nA, nB, idx, score(ir2, nodeIds | aS(tuple))]

In [62]:
#export
def traj(ir, maxSteps=20, nodeIds=None):
    sc = score(ir, nodeIds); aux = None; auxs = []; irs = []
    maxSteps = maxSteps if maxSteps is not None else int(1e10)
    for i in range(maxSteps):
        print(f"\rVirtually moving files and folders around. Step {i} of total max steps ({maxSteps})...", end="")
        ir, aux = optimize(ir, nodeIds)
        if aux[3] > sc: break
        irs.append(ir); auxs.append(aux); sc = aux[3]
    print(); return irs, auxs

In [11]:
traj(ir, nodeIds=tuple(applyCl.nodeIds()))[1] | display(None)

244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732   1fee395d9c928cf2d816b99a7c6e08d297e6ed4a1fc90932a0d67e9b   0    1.0822664696035837    
244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732   1fee395d9c928cf2d816b99a7c6e08d297e6ed4a1fc90932a0d67e9b   1    1.0821433302210242    
244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732   1fee395d9c928cf2d816b99a7c6e08d297e6ed4a1fc90932a0d67e9b   8    0.257887409190956     
244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732   1fee395d9c928cf2d816b99a7c6e08d297e6ed4a1fc90932a0d67e9b   13   0.2563133261101207    
244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732   1fee395d9c928cf2d816b99a7c6e08d297e6ed4a1fc90932a0d67e9b   2    0.25219192530965784   
244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732   1fee395d9c928cf2d816b99a7c6e08d297e6ed4a1fc90932a0d67e9b   12   0.2496689609745758    
244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732   1fee395d9c928cf2d816b99a7c6e08d297e6ed4a1fc90932a0d67e9b   9    0

In [27]:
#export
def collapse(it):
    a, b = it | rows(0, -1); c = [a[0], b[1], a[2], b[3]]
    return [] if c[0] == c[1] else [c]
def traj2(ir, traj): # just looks up the file names really, no processing involved
    idx2FileName = ir | apply(lambda arr: [arr[0], arr[2]]) | toDict()
    a = traj | groupBy(2) | filt(lambda x: len(x) > 1).split() | (apply(collapse)) + iden() | joinStreams(2) | deref()
    return a | lookup(idx2FileName, 2) | deref()
def moveFile(fileName:str, sourceNodeId:str, destNodeId:str, timeout=60): # old, slow, corrupted version
    """Moves file from the current node to the destination node. Usually executed on other nodes than the driver node"""
    fn = os.path.expanduser(fileName); dirname = os.path.dirname(fn)
    applyCl.cmd(f"mkdir -p {dirname}", nodeIds=[destNodeId]); applyCl.cmd(f"rm -f {fn}", nodeIds=[destNodeId])
    for chunk in cat(fn, False, True): [destNodeId] | applyCl.aS(lambda: chunk >> file(fn), timeout=timeout) | deref()
    None | cmd(f"rm {fn}") | deref(); return "ok1"
def moveFile(fn:str, sourceN:str, destN:str, timeout=60): # runs on dest node
    fn = os.path.expanduser(fn); dirname = os.path.dirname(fn)
    windows = [sourceN] | applyCl.aS(lambda: range(os.path.getsize(fn)) | batched(settings.cli.cat.chunkSize, True) | apply("[x.start, x.stop]") | deref()) | cut(1) | item() | deref()
    for chunk in [[sourceN]*len(windows), windows] | transpose() | ~applyCl(lambda sB,eB: cat(fn, False, sB=sB, eB=eB), pre=True, prefetch=20) | cut(1): chunk >> file(fn)
    [sourceN] | applyCl.aS(lambda: None | cmd(f"rm {fn}") | deref()) | deref()
    return "ok file"
def moveFF(ff:str, sourceN:str, destN:str, timeout=60): # runs on dest node
    """Moves file or folder from the current node to the destination node"""
    ff = os.path.expanduser(ff); isfile = [sourceN] | applyCl.aS(lambda: os.path.isfile(ff)) | cut(1) | item()
    if isfile: return moveFile(ff, sourceN, destN, timeout)
    [sourceN] | applyCl.aS(lambda: ff | getFilesInFolder | deref()) | cut(1) | item() | apply(aS(moveFile, sourceN, destN, timeout)) | deref()
    None | cmd(f"rm -rf {ff}") | ignore(); return f"ok folder: {ff}"
def moveAll(tr, timeout=60):
    groups = tr | groupBy(1) | deref() # grouping by destination
    with ray.progress(len(groups), "Moving files around") as rp:
        def process(idx, arrs): # processing requests for dest node
            for i, arr in enumerate(arrs): # moveFF(fn, a, b)
                [[arr[1], arr]] | ~applyCl(lambda a,b,fn,sc: moveFF(fn, a, b), pre=True, timeout=timeout) | deref() # move 1 file on dest node
                rp.update.remote(idx, (i+1)/len(arrs))
        groups | insertIdColumn() | ~applyTh(process, timeout=None) | deref()
    # tr | apply(lambda arr: [arr[0], arr]) | ~applyCl(lambda a,b,fn,sc: moveFF(fn, b), pre=True, timeout=timeout) | deref() # old version without progress bar
def balanceFolder(base, audit=False, maxSteps=1000, timeout=60): # currently executing each move step serially, will change in the future if it's too slow
    loadTestGuard(); applyCl.cmd(f"mkdir -p {base}"); ir = getIr(base)
    tr = traj2(ir, traj(ir, maxSteps, tuple(applyCl.nodeIds()))[1]); return tr if audit else moveAll(tr)
def decommissionFolderTraj(base:str, nAs:List[str]): # nAs are the ones to decommission
    ir = getIr(base); targetNodes = applyCl.nodeIds() | ~inSet(nAs) | repeatFrom(); irs = []; auxs = []
    seis = ir | inSet(nAs, 1) | cut(0, 1) | ~apply(lambda idx, startNode: [startNode, next(targetNodes), idx, 0]) | deref() # List[start, end, index, score]
    for sei in seis: ir = move(ir, *sei[:3]); irs.append(ir); auxs.append(sei)
    return irs, auxs
def decommissionFolder(base:str, nAs:List[str], audit=False, maxSteps=1000, timeout=60):
    loadTestGuard(); irs, auxs = decommissionFolderTraj(base, nAs)
    irs2, auxs2 = traj(irs[-1] if len(irs) > 0 else getIr(base), maxSteps, applyCl.nodeIds() | ~inSet(nAs) | aS(tuple))
    irs = [*irs, *irs2]; auxs = [*auxs, *auxs2]
    if len(irs) == 0: return
    tr = traj2(irs[-1], auxs); return tr if audit else moveAll(tr)

In [67]:
applyCl.cmd(f"rm -r {base}"); applyCl.cmd(f"mkdir -p {base}")
torch.loglinspace(10, 1e5, 100).int() | apply(lambda x: "x"*x) | insertIdColumn() | ~apply(lambda idx, contents: contents | file(f"{base}/{idx}.txt")) | deref();

In [68]:
None | applyCl.aS(lambda: ls(base) | apply(os.path.getsize) | toSum()) | deref()

[['1fee395d9c928cf2d816b99a7c6e08d297e6ed4a1fc90932a0d67e9b', 0],
 ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732', 1125606],
 ['ef95788f939f2409d731d963c3ea9281aa1b42f3a091faebc42f65bc', 0],
 ['bb09a2d34473af60b2274afc49b5b7f8ee11255bd7faf58b48c89dd1', 0]]

In [69]:
balanceFolder(base, False, 1000)
None | applyCl.aS(lambda: ls(base) | apply(os.path.getsize) | toSum()) | iden() & (cut(1) | toSum()) | deref()

applyCl has not load-tested your system yet, so running
the requested operation might be unbalanced on the cluster (some
nodes finish before others, wasting computational time). Would you
like to perform a load test now? Should take 1-2 minutes. Y/n:  y


Virtually moving files and folders around. Step 96 of total max steps (1000)...


[[['1fee395d9c928cf2d816b99a7c6e08d297e6ed4a1fc90932a0d67e9b', 781519],
  ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732', 198262],
  ['ef95788f939f2409d731d963c3ea9281aa1b42f3a091faebc42f65bc', 83022],
  ['bb09a2d34473af60b2274afc49b5b7f8ee11255bd7faf58b48c89dd1', 62803]],
 1125606]

In [70]:
assert None | applyCl.aS(lambda: ls(base) | apply(os.path.getsize) | toSum()) | cut(1) | toProd() > 0
torch.loglinspace(10, 1e5, 20).int() | apply(lambda x: "x"*x) | insertIdColumn() | ~apply(lambda idx, contents: contents | file(f"{base}/{idx+100}.txt")) | deref();
balanceFolder(base)
assert None | applyCl.aS(lambda: ls(base) | apply(os.path.getsize) | toSum()) | cut(1) | toProd() > 0
None | applyCl.aS(lambda: ls(base) | apply(os.path.getsize) | toSum()) | iden() & (cut(1) | toSum()) | deref()

Virtually moving files and folders around. Step 21 of total max steps (1000)...


[[['1fee395d9c928cf2d816b99a7c6e08d297e6ed4a1fc90932a0d67e9b', 950715],
  ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732', 237232],
  ['ef95788f939f2409d731d963c3ea9281aa1b42f3a091faebc42f65bc', 83022],
  ['bb09a2d34473af60b2274afc49b5b7f8ee11255bd7faf58b48c89dd1', 114944]],
 1385913]

In [71]:
decommissionFolder(base, applyCl.nodeIds()[:2])
None | applyCl.aS(lambda: ls(base) | apply(os.path.getsize) | toSum()) | iden() & (cut(1) | toSum()) | deref()

Virtually moving files and folders around. Step 999 of total max steps (1000)...


[[['1fee395d9c928cf2d816b99a7c6e08d297e6ed4a1fc90932a0d67e9b', 0],
  ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732', 0],
  ['ef95788f939f2409d731d963c3ea9281aa1b42f3a091faebc42f65bc', 756348],
  ['bb09a2d34473af60b2274afc49b5b7f8ee11255bd7faf58b48c89dd1', 629565]],
 1385913]

## Files integrity check

In [1]:
from k1lib.imports import *
base = "~/ssd2/test"; applyCl.cmd(f"rm -rf {base}"); None | cmd(f"mkdir -p {base}")
def genBinary(size=1000): return None | cmd(f"cat /dev/urandom | head -{size}", text=False) | aS(b"".join)
range(100) | apply(lambda i: genBinary(10000) | file(f"{base}/{i}.pth")) | deref();
con1 = ls(base) | iden() & apply(cat(text=False)) | transpose() | deref() | sort(0, False) | deref() | aS(k1.Wrapper)
applyCl.balanceFolder(base)
con2 = None | applyCl.aS(lambda: ls(base) | iden() & apply(cat(text=False)) | transpose() | deref()) | cut(1) | joinStreams() | sort(0, False) | deref() | aS(k1.Wrapper)
assert con1() == con2()

2023-05-24 13:45:01,706	INFO worker.py:1364 -- Connecting to existing Ray cluster at address: 192.168.1.35:6379...
2023-05-24 13:45:01,710	INFO worker.py:1544 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
(<lambda> pid=590423, ip=192.168.1.57) 2023-05-24 13:45:05,772	INFO worker.py:1364 -- Connecting to existing Ray cluster at address: 192.168.1.35:6379...
(<lambda> pid=269790, ip=192.168.1.53) 2023-05-24 13:45:05,782	INFO worker.py:1364 -- Connecting to existing Ray cluster at address: 192.168.1.35:6379...
(<lambda> pid=3088074, ip=192.168.1.43) 2023-05-24 13:45:05,816	INFO worker.py:1364 -- Connecting to existing Ray cluster at address: 192.168.1.35:6379...
(<lambda> pid=2576004) 2023-05-24 13:45:05,930	INFO worker.py:1364 -- Connecting to existing Ray cluster at address: 192.168.1.35:6379...


Virtually moving files and folders around. Step 82 of total max steps (10000000000)...


In [73]:
None | applyCl.aS(lambda: ls(base) | apply(os.path.getsize) | toSum()) | deref()

[['1fee395d9c928cf2d816b99a7c6e08d297e6ed4a1fc90932a0d67e9b', 371339],
 ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732', 96900],
 ['ef95788f939f2409d731d963c3ea9281aa1b42f3a091faebc42f65bc', 36535],
 ['bb09a2d34473af60b2274afc49b5b7f8ee11255bd7faf58b48c89dd1', 36523]]

## Linear scale file sizes

In [28]:
base = "~/repos/labs/k1lib/k1lib/cli/test/applyCl.balance"; #base = "~/ssd2/test"
applyCl.cmd(f"rm -r {base}"); applyCl.cmd(f"mkdir -p {base}")
torch.linspace(1e4, 1e5, 20).int() | apply(lambda x: "x"*x) | insertIdColumn() | ~apply(lambda idx, contents: contents | file(f"{base}/{idx}.txt")) | deref();
balanceFolder(base); None | applyCl.aS(lambda: ls(base) | apply(os.path.getsize) | toSum()) | deref()

[['51306d246cd47ada0addf519f27c0d313953d20aa3fe9e2080f2bb0d', 521059],
 ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732', 301581],
 ['ef95788f939f2409d731d963c3ea9281aa1b42f3a091faebc42f65bc', 124212],
 ['bb09a2d34473af60b2274afc49b5b7f8ee11255bd7faf58b48c89dd1', 153159]]

In [29]:
torch.linspace(1e4, 1e5, 20).int() | apply(lambda x: "x"*x) | insertIdColumn() | ~apply(lambda idx, contents: contents | file(f"{base}/{idx+20}.txt")) | deref();
balanceFolder(base); None | applyCl.aS(lambda: ls(base) | apply(os.path.getsize) | toSum()) | deref()

[['51306d246cd47ada0addf519f27c0d313953d20aa3fe9e2080f2bb0d', 1079484],
 ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732', 580533],
 ['ef95788f939f2409d731d963c3ea9281aa1b42f3a091faebc42f65bc', 253161],
 ['bb09a2d34473af60b2274afc49b5b7f8ee11255bd7faf58b48c89dd1', 286844]]

In [30]:
None | applyCl.aS(lambda: ls(base) | apply(os.path.getsize) | toSum()) | cut(1) | toSum() | deref()

2200022

In [31]:
torch.linspace(1e4, 1e5, 20).int() | toSum() | op()*2+40

tensor(2200022)

In [32]:
None | applyCl.aS(lambda: ls(base)) | deref()

[['51306d246cd47ada0addf519f27c0d313953d20aa3fe9e2080f2bb0d',
  ['/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance/23.txt',
   '/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance/32.txt',
   '/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance/12.txt',
   '/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance/19.txt',
   '/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance/2.txt',
   '/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance/35.txt',
   '/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance/0.txt',
   '/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance/29.txt',
   '/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance/6.txt',
   '/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance/10.txt',
   '/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance/36.txt',
   '/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance/34.txt',
   '/home/kelvin/repos/labs/k1lib/k1lib/cli/test/appl

Looks about right.

# Advanced split file

In [1]:
from k1lib.imports import *
base = "/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.advSplit"
fn = f"{base}/a.txt"
def reset(n=10000):
    applyCl.cmd(f"rm -r {base}"); applyCl.cmd(f"mkdir -p {base}")
    applyCl.nodeIds() | head(2) | applyCl.aS(lambda: "0123456789\n"*n | file(fn)) | deref()
def status(): return None | applyCl.aS(lambda: fn | (tryout(0) | cat(text=False) | shape(0))) | deref()
nodeIds = applyCl.nodeIds(); reset(); status()

2023-05-15 13:49:11,461	INFO worker.py:1364 -- Connecting to existing Ray cluster at address: 192.168.1.35:6379...
2023-05-15 13:49:11,480	INFO worker.py:1544 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


[['867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0', 110001],
 ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732', 110001],
 ['4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d', 0]]

In [2]:
nse = [applyCl.nodeIds()[:2], [0, 1234], [4567, 12345]] | transpose() | apply(lambda arr: [arr[0], arr[1:]]) | deref(); nse

[['867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0', [0, 4567]],
 ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732', [1234, 12345]]]

In [3]:
#export
def a_transfer(fn, nse, nodeB, rpF:callable=iden()):
    """Transfers a lot of blocks from a bunch of nodes to nodeB. Does not delete from those node though

nse = List[nodeAId, [sB, eB]]

Runs on driver process, blocks, so better use applyTh outside of this

:param rpF: ray progress function"""
    blockSize = settings.cli.cat.chunkSize
    def inner():
        totalBytes = nse | cut(1) | ~apply(lambda x,y:y-x) | toSum(); currentByte = 0
        for chunk in nse | ~apply(lambda x, y: range(x, y) | batched(blockSize, True) | apply("[x.start, x.stop]"), 1) | ungroup() | deref()\
            | ~applyCl(lambda sB, eB: cat(fn, False, sB=sB, eB=eB), pre=True, timeout=None, prefetch=20) | cut(1):
            chunk >> file(fn); currentByte += len(chunk); rpF(currentByte/totalBytes)
    [nodeB] | applyCl.aS(inner, timeout=None) | item()

In [4]:
%%time
reset()
a_transfer(fn, nse, applyCl.nodeIds()[2])
assert None | applyCl.aS(lambda: cat(fn, False) | shape(0)) | deref() | op()[2][1] == 4567+11111

CPU times: user 18.8 ms, sys: 0 ns, total: 18.8 ms
Wall time: 155 ms


In [5]:
#export
def decommission(fn:str, nodeAs:List[str], nodeBs:List[str], rS):
    """Spreads out a particular file in nodeAs to all nodeBs, to prepare
to decomission nodeAs. The 2 sets should be mutually exclusive

:param rS: instance of refineSeek"""
    nodeAs, nodeBs = [nodeAs, nodeBs] | deref()
    if len(nodeAs) == 0: return
    if len(nodeBs) == 0: raise Exception("Unsupported configuration! Trying to move data from A+B to C+D. Has to have some shared nodes, like moving data from A+B+C to B+C+D. This is not a fundamental limitation, but just can't be done with the current architecture. Might be fixed in the future.")
    # some initial metadata
    nodeIds = applyCl.nodeIds(); nodeId_cpu = loadTestGuard(False).items() | deref(); nodeId2Cpu = nodeId_cpu | toDict()
    ws = nodeId_cpu | inSet(nodeBs, 0) | cut(1) | deref() # weights to split files on nodeAs into
    # splitting file on nodeAs into chunks first, to plan things out
    a = nodeAs | applyCl.aS(lambda: fn | splitSeek(ws=ws) | rS | window(2) | deref() | insertColumn(nodeBs) | insert(applyCl.nodeId()).all() | deref()) | cut(1) | joinStreams() | deref()
    # actually transferring chunks
    with ray.progress(a | groupBy(1) | shape(0), "Decommissioning") as rp:
        c = b = a | groupBy(1, True) | apply(iden() + apply(lambda arr: [arr[0], arr[1:]]) | reverse() | insert(fn)) | deref()
        enumerate(c) | applyTh(~aS(lambda idx, e: a_transfer(*e, rpF=aS(lambda p: ray.get(rp.update.remote(idx, p))))), timeout=None) | deref()
    # deleting files from nodeAs
    nodeAs | applyCl.aS(lambda: None | cmd(f"rm -rf {fn}") | ignore()) | deref()

In [6]:
reset(); status() | display()
decommission(fn, *applyCl.nodeIds() | splitW(1, 2))
assert status() | ~head(1) | cut(1) | toProd() == 6723517725

867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0   110001   
244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732   110001   
4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d   0        


In [7]:
status()

[['867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0', 0],
 ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732', 183327],
 ['4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d', 36675]]

In [8]:
nAs = ["867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0", "244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732"]
nBs = ["4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d"]
ns = [*nAs, *nBs]

In [25]:
#export
ranges2Seeks = apply("[x.start, x.stop]") | joinStreams() | aS(set) | sort(None) | deref()
def spreadOut(fn:str, nAs:List[str], nBs:List[str], rS):
    """Spreads out a file from nodes A to B, where B fully contains A (no decomissioning).
A and B should be mutually exclusive. Initial nodes are A, final nodes are A + B"""
    nAs, nBs = [nAs, nBs] | deref(); rS.fn = fn
    if len(nBs) == 0: return # no need to spread out
    nBs | applyCl.aS(lambda: None | cmd(f"mkdir -p {os.path.dirname(fn)}") | deref(), timeout=None) | deref()
    nBs | applyCl.aS(lambda: None | cmd(f"rm -rf {fn}") | deref(), timeout=None) | deref()
    # some initial metadata
    nodeIds = applyCl.nodeIds(); nodeId_cpu = loadTestGuard(False).items() | deref(); nodeId2Cpu = nodeId_cpu | toDict()
    sizes = nAs | applyCl.aS(lambda: os.path.getsize(fn) if os.path.exists(fn) else 0) | deref(); totalSize = sizes | cut(1) | toSum()
    ns = [*nAs, *nBs]; totalCpu = ns | lookup(nodeId2Cpu) | toSum(); bytePerCpu = totalSize/totalCpu; wsB = nBs | lookup(nodeId2Cpu) | deref()
    # prepares segments and metadata, List[nodeId, [sB, eB]], where sB and eB are the ranges of nAs that they're willing to share
    sizePost = sizes | ~apply(lambda idx, size: [idx, nodeId2Cpu[idx]/totalCpu*totalSize/size]) | deref() # size fraction to retain
    invalidNodes = sizePost | ~filt(lambda x: 0 <= x <= 1, 1) | cut(0) | deref()
    if len(invalidNodes) > 0: raise Exception(f"Unsupported configuration! These nodes have too little data to share: {invalidNodes}. This couldn't have happen using applyCl alone. Data is not corrupted, but you'll have to combine data from all files into 1 and spread them back out again.")
    inter = sizePost | ~apply(lambda idx, x: [idx, [x, 1-x]]) | applyCl(lambda ws: fn | splitSeek(ws=ws) | rS | ~head(1), pre=True) | deref() | filt(~aS(lambda x,y: y-x>0), 1) | deref() # filter at the end to eliminate files that don't want to share anything (x == y)
    # actually transferring data to new nodes
    meta = inter | apply(~aS(range) | splitW(*wsB) | ranges2Seeks | apply(lambda x: splitSeek.backward(fn, x)) | deref() | rS | window(2) | deref() | apply(wrapList()) | insertColumn(nBs), 1) | ungroup(False) | groupBy(1, True) | deref()
    with ray.progress(len(meta), "Transferring data to new nodes") as rp:
        meta | insertIdColumn(True) | applyTh(~aS(lambda idx, nB, nse: a_transfer(fn, nse, nB, rpF=aS(lambda p: ray.get(rp.update.remote(idx, p))))), timeout=24*3600) | deref()
    # truncates the files in nAs nodes
    inter | ~apply(lambda idx,se: [idx,se[0]]) | applyCl(lambda sB: open(fn, 'a').truncate(sB), pre=True, timeout=None) | deref()

In [26]:
#export
def balanceFile(fn:str, nAs:List[str]=None, nBs:List[str]=None, rS=None):
    fn = os.path.expanduser(fn); rS = rS or refineSeek(); rS.injectFn(fn); loadTestGuard()
    if nAs is None: nAs = None | applyCl.aS(lambda: os.path.exists(fn)) | filt(op(), 1) | cut(0) | deref()
    if nBs is None: nBs = applyCl.nodeIds()
    decommission(fn, *nAs | inSet(nBs).split() | reverse(), rS)
    spreadOut(fn, *nBs | inSet(nAs).split(), rS)

In [27]:
reset(10000); "0123456789\n"*5000 >> file(fn); status()

[['867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0', 110001],
 ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732', 165002],
 ['4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d', 0]]

In [28]:
balanceFile(fn); status()

[['867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0', 68750],
 ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732', 137501],
 ['4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d', 68752]]

In [18]:
balanceFile(fn, nBs=["867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0", "4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d"]); status()

[['867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0', 137500],
 ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732', 0],
 ['4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d', 137503]]

In [29]:
balanceFile(fn, nBs=["4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d"]); status()

[['867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0', 0],
 ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732', 0],
 ['4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d', 275003]]

In [20]:
balanceFile(fn); status()

[['867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0', 68754],
 ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732', 137509],
 ['4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d', 68740]]

## File integrity check

In [ ]:
from k1lib.imports import *
base = "~/ssd2/test"; fn = f"{base}/a.txt"; applyCl.cmd(f"rm -rf {base} && mkdir -p {base}")
def genBinary(size=1000): return None | cmd(f"cat /dev/urandom | head -{size}", text=False)
genBinary(1_000_000) | apply(k1.encode) | sort(None, False) | file(fn)
con1 = cat(fn) | sort(None, False) | aS(list) | aS(k1.Wrapper)
applyCl.balanceFile(fn)
con2 = applyCl.cat(fn, filt(op()) | deref()) | joinStreams() | sort(None, False) | aS(list) | aS(k1.Wrapper)
applyCl.decommissionFile(fn, ["2fd74a5de08d96d323420c575ae514fdcb58f2987b7b621bfde3485c"])
con3 = applyCl.cat(fn, filt(op()) | deref()) | joinStreams() | sort(None, False) | aS(list) | aS(k1.Wrapper)
assert con1() == con2(); assert con2() == con3()

# diskScan

Tries to scan the disk of all nodes to figure out where are the distributed files and folders.

In [4]:
from k1lib.imports import *
base = "/home/kelvin/ssd2"

In [7]:
#export
def diskScan1(base:str) -> List[str]: # like ls(), but returns files and folders that appear at least on 2 nodes
    isdir, base = base.split("\ue000")
    if not isdir: return []
    return None | applyCl.aS(lambda: base | (tryout([]) | ls() | apply(os.path.isdir) & iden() | transpose() | ~apply(lambda x,y: f"{x*1}\ue000{y}")) | deref(), timeout=60) | cut(1) | joinStreams() | count() | filt(op()>1, 0) | cut(1) | deref()
def diskScan2(base:str) -> Tuple[List[str], List[str]]: # returns list of distributed folders and list of distributed files
    dFolders = []; folders, files = diskScan1(base) | op().split("\ue000").all() | toInt(0) | filt(op(), 0).split() | (join("\ue000")).all(2) | deref()
    # print("2--", folders, files, base)
    for folder in folders:
        fol, fil = diskScan2(folder); dFolders.extend(fol); files.extend(fil)
        if len(fol) + len(fil) == 0: dFolders.append(folder) # no shared contents, must be a distributed folder
        else: files.extend(fil)
    # print("3--", [dFolders, files], base)
    return [dFolders, files]
def diskScan3(base:str): base = os.path.expanduser(base); return diskScan2(f"1\ue000{base}") | op().split("\ue000")[1].all(2) | deref()
def diskScan4(base:str, sortSize=True): # fully featured data
    folders, files = diskScan3(base)
    folders = [folders, None | applyCl.aS(lambda: folders | apply(lambda x: (x | getFolderSize) if os.path.exists(x) else 0) | deref(), timeout=60) | cut(1) | transpose()] | transpose() | deref()
    files   = [files,   None | applyCl.aS(lambda: files   | apply(lambda x: os.path.getsize(x)  if os.path.exists(x) else 0) | deref(), timeout=60) | cut(1) | transpose()] | transpose() | deref()
    post = apply(~sortF(toSum(), 1)) if sortSize else iden()
    return [folders, files] | wrapList() + filt(filt(op() > 0) | count() | shape(0) | (op() == 1), 1).split() | joinStreams() | apply(unique(0)) | post | deref()
def diskScan5(base:str, sortSize=True): # displays it in a nice format
    d4 = diskScan4(base, sortSize); lens = d4 | apply(len) | deref(); nodeNames = None | applyCl.aS(lambda: os.cpu_count()) | apply(op()[:5], 0) | apply('f"{x} thr"', 1) | join(", ").all() | deref(); nodeNames
    d5 = d4 | apply(~apply(lambda path, sizes: [path, sizes | toSum() | aS(fmt.size), sizes | apply(fmt.size)]) | insert(["-"*40, "-"*10, ["-"*12]*len(nodeNames)]) | insert(["", "", nodeNames])) | deref(); d5
    ws = d5 | shape(0).all() | deref()
    d6 = d5 | joinStreams() | cut(0, 1) & (cut(2) | pretty() | wrapList().all()) | transpose() | joinStreams().all() | splitW(*ws) | insert(["Path", "Total size", "Size on each node (node id and thread count)"]).all() | joinStreams() | pretty() | splitW(*ws | apply(op()+1)) | deref()
    explainers = ["\nA distributed folder is a folder that has many files and folders inside, but their names\nare all different from each other. It's managed by applyCl.balanceFolder()",
                  "\nA replicated file is a file that has been copied to multiple nodes. Size of all file\ncopies should be the same. It's managed by applyCl.replicateFile()",
                  "\nA distributed file is a file that has been split into multiple pieces and sent to other\nnodes. It's managed by applyCl.balanceFile()"]
    arr = [d6, ["Distributed folders", "Replicated files", "Distributed files"] | (aS(lambda x: [["-"*60, x, "-"*60] | join(" ")])).all()] | transpose() | permute(1, 0) | (joinStreams() | join("\n")).all() | wrapList() | insert(explainers, False) | transpose() | join("\n").all() | deref()
    [arr, lens] | transpose() | filt(op(), 1) | cut(0) | join("\n"*2) | wrapList() | stdout()

In [15]:
diskScan3(base)

[['/home/kelvin/ssd2/test',
  '/home/kelvin/ssd2/data/genome/go/release_geneontology_org',
  '/home/kelvin/ssd2/data/genome/genbank/ch1',
  '/home/kelvin/ssd2/data/genome/genbank/ch1.dat.gz',
  '/home/kelvin/ssd2/data/genome/RegulationFeatureActivity',
  '/home/kelvin/ssd2/data/genome/RegulationFeatureActivity.backup',
  '/home/kelvin/ssd2/data/genome/00-common_all.idx'],
 ['/home/kelvin/ssd2/data/genome/dummy.txt',
  '/home/kelvin/ssd2/data/genome/00-common_all.vcf',
  '/home/kelvin/ssd2/data/genome/00-All.vcf',
  '/home/kelvin/ssd2/data/genome/MotifFeatures/homo_sapiens.GRCh38.motif_features.gff',
  '/home/kelvin/ssd2/data/genome/MotifFeatures/homo_sapiens.GRCh38.motif_features.gff',
  '/home/kelvin/ssd2/data/genome/dummy.txt',
  '/home/kelvin/ssd2/data/genome/00-common_all.vcf',
  '/home/kelvin/ssd2/data/genome/00-All.vcf',
  '/home/kelvin/ssd2/data/genome/MotifFeatures/homo_sapiens.GRCh38.motif_features.gff',
  '/home/kelvin/ssd2/data/genome/MotifFeatures/homo_sapiens.GRCh38.motif_

In [5]:
diskScan4(base)

[[['/home/kelvin/ssd2/data/genome/RegulationFeatureActivity',
   [0, 7912834090, 4113489746, 4164314316]],
  ['/home/kelvin/ssd2/data/genome/go/release_geneontology_org',
   [0, 4172737915, 2071645117, 2107005131]],
  ['/home/kelvin/ssd2/data/genome/RegulationFeatureActivity.backup',
   [0, 552888466, 568878496, 600610083]],
  ['/home/kelvin/ssd2/data/genome/00-common_all.idx',
   [0, 671136833, 341738564, 0]],
  ['/home/kelvin/ssd2/data/genome/genbank/ch1.dat.gz',
   [0, 0, 25356744, 25356764]],
  ['/home/kelvin/ssd2/data/genome/genbank/ch1', [0, 0, 0, 0]]],
 [['/home/kelvin/ssd2/data/genome/dummy.txt', [0, 1101, 1101, 1101]]],
 [['/home/kelvin/ssd2/data/genome/00-All.vcf',
   [0, 65475018903, 32737509360, 32737509588]],
  ['/home/kelvin/ssd2/data/genome/MotifFeatures/homo_sapiens.GRCh38.motif_features.gff',
   [27927710370, 13963854708, 6981927367, 6981927374]],
  ['/home/kelvin/ssd2/data/genome/00-common_all.vcf',
   [0, 4707803470, 2353901811, 2353901831]],
  ['/home/kelvin/ssd2/te

In [17]:
diskScan5(base)

------------------------------------------------------------ Distributed folders ------------------------------------------------------------
Path                                                                                 Total size   Size on each node (node id and thread count)     
                                                                                                  ae5f4, 8 thr   244f4, 16 thr   f776e, 8 thr      
----------------------------------------                                             ----------   ------------   ------------    ------------      
/home/kelvin/ssd2/data/genome/RegulationFeatureActivity                              16.19 GB     4.11 GB        7.91 GB         4.16 GB           
/home/kelvin/ssd2/data/genome/go/release_geneontology_org                            8.35 GB      2.07 GB        4.17 GB         2.11 GB           
/home/kelvin/ssd2/data/genome/RegulationFeatureActivity.backup                       1.72 GB      568.88 MB      552.8

In [24]:
with k1.captureStdout() as out: diskScan5(base)
out() | apply(lambda x: f"   {x}") | insert("") | insert(".. code-block:: text") | file("../../docs/literals/diskScan.rst")

'../../docs/literals/diskScan.rst'

In [3]:
!../../export.py cli/_applyCl

Current dir: /home/kelvin/repos/labs/k1lib, /home/kelvin/repos/labs/k1lib/k1lib/cli/../../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.4
Uninstalling k1lib-1.4:
  Successfully uninstalled k1lib-1.4
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing requirements to k1lib.egg-info/requires.txt
writing top-level names to k1lib.egg-info/top_level.txt
w